# Description

**TODO**

# Modules

In [1]:
library(reticulate)
pd <- import("pandas")

# Settings

In [2]:
# reference panel
REFERENCE_PANEL <- "GTEX_V8"
# REFERENCE_PANEL = "1000G"

# prediction models
## mashr
EQTL_MODEL <- "MASHR"

# ## elastic net
# EQTL_MODEL = "ELASTIC_NET"

chromosome <- NULL

In [3]:
# Parameters
chromosome = 3
REFERENCE_PANEL = "GTEX_V8"
EQTL_MODEL = "MASHR"


In [4]:
paste0("Using reference panel: ", REFERENCE_PANEL)

[1] "Using reference panel: GTEX_V8"

In [5]:
paste0("Using eQTL model: ", EQTL_MODEL)

[1] "Using eQTL model: MASHR"

In [6]:
# chromosome must be provided as parameter
stopifnot(!is.null(chromosome))

# Paths

In [7]:
GENE_CORRS_DIR <- Sys.getenv("PHENOPLIER_PHENOMEXCAN_LD_BLOCKS_GENE_CORRS_DIR")
IRdisplay::display(GENE_CORRS_DIR)

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs"

In [8]:
INPUT_DIR <- file.path(GENE_CORRS_DIR, tolower(REFERENCE_PANEL), tolower(EQTL_MODEL), "by_chr")
IRdisplay::display(INPUT_DIR)

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/gtex_v8/mashr/by_chr"

In [9]:
INPUT_FILE <- file.path(INPUT_DIR, paste0("gene_corrs-chr", chromosome, ".pkl"))
IRdisplay::display(INPUT_FILE)
stopifnot(file.exists(INPUT_FILE))

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/gtex_v8/mashr/by_chr/gene_corrs-chr3.pkl"

In [10]:
OUTPUT_DIR <- file.path(INPUT_DIR, "corrected_positive_definite")
IRdisplay::display(OUTPUT_DIR)
dir.create(OUTPUT_DIR, recursive = TRUE, showWarnings = FALSE)

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/gtex_v8/mashr/by_chr/corrected_positive_definite"

In [11]:
OUTPUT_FILE <- file.path(OUTPUT_DIR, paste0("gene_corrs-chr", chromosome, ".pkl"))
IRdisplay::display(OUTPUT_FILE)
if (file.exists(OUTPUT_FILE)) {
  IRdisplay::display("Output file exists, it will be overwritten")
}

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/gtex_v8/mashr/by_chr/corrected_positive_definite/gene_corrs-chr3.pkl"

[1] "Output file exists, it will be overwritten"

# Functions

In [12]:
# taken from https://www.r-bloggers.com/2013/08/correcting-a-pseudo-correlation-matrix-to-be-positive-semidefinite/
# TODO: add documentation
CorrectCM <- function(CM, p = 0) {
  n <- dim(var(CM))[1L]
  E <- eigen(CM)
  CM1 <- E$vectors %*% tcrossprod(diag(pmax(E$values, p), n), E$vectors)
  Balance <- diag(1 / sqrt(diag(CM1)))
  CM2 <- Balance %*% CM1 %*% Balance
  return(CM2)
}

In [13]:
is_positive_definite <- function(data) {
  eigenvalues <- eigen(data)$values
  nonpositive_eigenvalues <- eigenvalues[eigenvalues <= 0]

  if (length(nonpositive_eigenvalues) > 0) {
    IRdisplay::display("We need to correct the data and make the matrix positive definite")
    return(FALSE)
  } else {
    IRdisplay::display("Matrix is already positive definite!")
    return(TRUE)
  }
}

# Load data

In [14]:
gene_corrs <- pd$read_pickle(INPUT_FILE)

In [15]:
dim(gene_corrs)

[1] 393 393

In [16]:
head(gene_corrs[1:10, 1:10])

,ENSG00000134121,ENSG00000091181,ENSG00000144455,ENSG00000170364,ENSG00000150995,ENSG00000134107,ENSG00000134109,ENSG00000180914,ENSG00000196220,ENSG00000163719
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000134121,1.000000000,-0.005292137,0.03000655,-0.02264608,-0.042667080,0.095730848,-0.042436975,0.00000000,0.00000000,0.0000000
ENSG00000091181,-0.005292137,1.000000000,0.06407092,-0.05455147,0.073573178,-0.024861424,-0.035525412,0.00000000,0.00000000,0.0000000
ENSG00000144455,0.030006554,0.064070923,1.00000000,0.23056040,0.038559521,0.022396071,0.105842350,0.05855223,0.04639374,0.0000000
ENSG00000170364,-0.022646076,-0.054551471,0.23056040,1.00000000,-0.037986584,0.039735622,-0.067866315,0.06449193,-0.15825002,0.0000000
ENSG00000150995,-0.042667080,0.073573178,0.03855952,-0.03798658,1.000000000,-0.007345318,-0.028169573,0.00820978,0.04093686,-0.1082528
ENSG00000134107,0.095730848,-0.024861424,0.02239607,0.03973562,-0.007345318,1.000000000,0.005379158,0.07821121,0.10829504,-0.1113816


# Check positive definiteness

In [17]:
is_positive_definite(gene_corrs)

[1] "We need to correct the data and make the matrix positive definite"

[1] FALSE

In [18]:
# see eigenvalues
eigenvalues <- eigen(gene_corrs)$values

In [19]:
nonpositive_eigenvalues <- eigenvalues[eigenvalues <= 0]
IRdisplay::display(length(nonpositive_eigenvalues))
IRdisplay::display(nonpositive_eigenvalues)

[1] 33

[1] -0.02435129 -0.03192894 -0.06715815 -0.09547102 -0.11076508 -0.11553767
 [7] -0.14851162 -0.17922089 -0.20675717 -0.24503213 -0.32637233 -0.39420240
[13] -0.41630304 -0.46647937 -0.56873050 -0.57651120 -0.66883253 -0.68650090
[19] -0.76320608 -0.76826429 -0.84742222 -0.94716193 -1.05594220 -1.11520885
[25] -1.27053166 -1.28093201 -1.41426587 -1.51038246 -1.78547293 -1.81449589
[31] -2.02964848 -2.46435557 -4.03665238

In [20]:
if (length(eigenvalues[eigenvalues <= 0]) == 0) { quit() }

# Make matrix positive definite if needed

In [21]:
gene_corrs_corrected <- CorrectCM(gene_corrs, 1)

In [22]:
dimnames(gene_corrs_corrected)[[1]] <- rownames(gene_corrs)

In [23]:
dimnames(gene_corrs_corrected)[[2]] <- colnames(gene_corrs)

In [24]:
gene_corrs_corrected <- as.data.frame(gene_corrs_corrected)

In [25]:
dim(gene_corrs_corrected)

[1] 393 393

# Check positive definiteness of corrected matrix

In [26]:
is_positive_definite(gene_corrs_corrected)

[1] "Matrix is already positive definite!"

[1] TRUE

In [27]:
# see eigenvalues
eigenvalues <- eigen(gene_corrs_corrected)$values

In [28]:
nonpositive_eigenvalues <- eigenvalues[eigenvalues <= 0]
IRdisplay::display(length(nonpositive_eigenvalues))
IRdisplay::display(nonpositive_eigenvalues)

[1] 0

numeric(0)

In [29]:
stopifnot(length(eigenvalues[eigenvalues <= 0]) == 0)

In [30]:
# quick and visual comparison of the two matrices
IRdisplay::display(head(gene_corrs[1:10, 1:10]))
IRdisplay::display(head(gene_corrs_corrected[1:10, 1:10]))

,ENSG00000134121,ENSG00000091181,ENSG00000144455,ENSG00000170364,ENSG00000150995,ENSG00000134107,ENSG00000134109,ENSG00000180914,ENSG00000196220,ENSG00000163719
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000134121,1.000000000,-0.005292137,0.03000655,-0.02264608,-0.042667080,0.095730848,-0.042436975,0.00000000,0.00000000,0.0000000
ENSG00000091181,-0.005292137,1.000000000,0.06407092,-0.05455147,0.073573178,-0.024861424,-0.035525412,0.00000000,0.00000000,0.0000000
ENSG00000144455,0.030006554,0.064070923,1.00000000,0.23056040,0.038559521,0.022396071,0.105842350,0.05855223,0.04639374,0.0000000
ENSG00000170364,-0.022646076,-0.054551471,0.23056040,1.00000000,-0.037986584,0.039735622,-0.067866315,0.06449193,-0.15825002,0.0000000
ENSG00000150995,-0.042667080,0.073573178,0.03855952,-0.03798658,1.000000000,-0.007345318,-0.028169573,0.00820978,0.04093686,-0.1082528
ENSG00000134107,0.095730848,-0.024861424,0.02239607,0.03973562,-0.007345318,1.000000000,0.005379158,0.07821121,0.10829504,-0.1113816


,ENSG00000134121,ENSG00000091181,ENSG00000144455,ENSG00000170364,ENSG00000150995,ENSG00000134107,ENSG00000134109,ENSG00000180914,ENSG00000196220,ENSG00000163719
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000134121,1.0000000000,-0.015838150,0.0005508348,0.008641984,-0.015263570,0.046854446,-1.789286e-02,0.007867298,0.009243721,-0.0069367892
ENSG00000091181,-0.0158381503,1.000000000,0.0119208519,-0.007808076,0.047419729,-0.011580197,-1.157882e-02,-0.002667315,0.006272357,-0.0075740714
ENSG00000144455,0.0005508348,0.011920852,1.0000000000,0.095976725,0.012480433,0.027703962,3.626678e-02,0.049446034,0.004202449,-0.0099385792
ENSG00000170364,0.0086419839,-0.007808076,0.0959767253,1.000000000,-0.013972284,0.019488312,7.626046e-03,0.036921287,-0.057585165,0.0005127878
ENSG00000150995,-0.0152635705,0.047419729,0.0124804329,-0.013972284,1.000000000,0.004382408,-5.473936e-03,0.001664691,0.015162669,-0.0340921263
ENSG00000134107,0.0468544462,-0.011580197,0.0277039623,0.019488312,0.004382408,1.000000000,1.112579e-05,0.043542628,0.033606797,-0.0334050645


Both matrices should "look" similar. We are not interested in perfectly accurate correlation values (they are already inaccurate).

# Save

In [31]:
py_save_object(gene_corrs_corrected, OUTPUT_FILE)